<a href="https://colab.research.google.com/github/PatrickD321/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,184 kB]
Hit:12 http://ppa.launchpad.net/cra

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-07 22:42:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.69MB/s    in 0.2s    

2021-06-07 22:42:22 (5.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27288431| R33UPQQUZQEM8|B005T4ND06|     400024643|Yoga for Movement...|       Video DVD|          5|            3|          3|   N|                Y|This was a gift f...|This was a gift f...| 2015-08-31|
|         US|   13722556|R3IKTNQQPD9662|B004EPZ070|     685335564|  Something Borrowed|       Video DVD|          5|    

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df = df.toDF('marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date')

In [7]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R33UPQQUZQEM8|          5|            3|          3|   N|                Y|
|R3IKTNQQPD9662|          5|            0|          0|   N|                Y|
|R3U27V5QMCP27T|          5|            1|          1|   N|                Y|
|R2TOH2QKNK4IOC|          5|            0|          1|   N|                Y|
|R2XQG5NJ59UFMY|          5|            0|          0|   N|                Y|
|R1N1KHBRR4ZTX3|          5|            0|          0|   N|                Y|
|R3OM9S0TCBP38K|          5|            0|          0|   N|                Y|
|R1W4S949ZRCTBW|          5|            0|          0|   N|                Y|
|R18JL1NNQAZFV2|          5|            0|          0|   N|                Y|
|R1LP6PR06OPYUX|          4|            0|          0|   N|     

In [14]:
# Filter table for total_votes > 20
vine_filter_df = vine_df.filter("total_votes>20").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine","verified_purchase"])
vine_filter_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|
|R12Q1NO1HI9PP4|          1|            4|         86|   N|                N|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|
| R6TNONSNFM82I|          1|            0|         28|   N|                Y|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|
| RUNB838XKMB59|          1|            8|         23|   N|                N|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|
|R2WFUSIPJ3X0FR|          1|            9|         53|   N|                N|
| RONWXF0J85CRR|          5|           42|         50|   N|     

In [48]:
# Ratio of helpful_votes to total_votes
vine = vine_filter_df.withColumn("ratio",  (vine_filter_df["helpful_votes"] / vine_filter_df["total_votes"]))
vine.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|               ratio|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|  0.8205128205128205|
|R12Q1NO1HI9PP4|          1|            4|         86|   N|                N|0.046511627906976744|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|  0.9459459459459459|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|  0.9858156028368794|
| R6TNONSNFM82I|          1|            0|         28|   N|                Y|                 0.0|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|  0.7619047619047619|
| RUNB838XKMB59|          1|            8|         23|   N|                N| 0.34782608695652173|
|R3I8GORXG

In [50]:
# Filter for ratio greater than 50%
vine_ratio = vine.filter("ratio >.5").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine","verified_purchase", "ratio"])
vine_ratio.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|0.8205128205128205|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|0.9459459459459459|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|0.9858156028368794|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|0.7619047619047619|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|0.9223300970873787|
| RONWXF0J85CRR|          5|           42|         50|   N|                N|              0.84|
|R3O7T74HRIXBE5|          5|           53|         55|   N|                Y|0.9636363636363636|
| RSU37FK0W4DPE|          4|  

In [56]:
# Filter ratio with vine paid votes
vine_Y = vine_ratio.filter(vine_ratio.vine == "Y")
vine_Y.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3PBQ665MBL8CP|          3|           66|         72|   Y|                N|0.9166666666666666|
|R3EMD30K7MNOZ0|          2|           20|         25|   Y|                N|               0.8|
|R2GF3ZLO2B0Y71|          5|          236|        243|   Y|                N|0.9711934156378601|
|R1KAQEESF0NUGV|          4|           27|         29|   Y|                N|0.9310344827586207|
|R1621B9AMVWI25|          2|           27|         29|   Y|                N|0.9310344827586207|
|R3FVGQVNBAZL8E|          3|           37|         44|   Y|                N|0.8409090909090909|
|R26ATTA765WHVX|          5|           21|         23|   Y|                N|0.9130434782608695|
| RMZYPFKES1ZK7|          4|  

In [57]:
# Filter for vine votes not paid
vine_N = vine_ratio.filter(vine_ratio.vine == "N")
vine_N.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|             ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R31EUR60FV3BU5|          4|           32|         39|   N|                N|0.8205128205128205|
|R3MDX24QDAT0OW|          4|           35|         37|   N|                N|0.9459459459459459|
|R2YVFZB4ESDT7D|          5|          139|        141|   N|                Y|0.9858156028368794|
|R2R8EWH1JUZCSI|          5|           16|         21|   N|                N|0.7619047619047619|
|R3I8GORXGEJL3R|          4|           95|        103|   N|                Y|0.9223300970873787|
| RONWXF0J85CRR|          5|           42|         50|   N|                N|              0.84|
|R3O7T74HRIXBE5|          5|           53|         55|   N|                Y|0.9636363636363636|
| RSU37FK0W4DPE|          4|  

In [60]:
# 5 star rating count
vine_5star = vine_df.filter(vine_df.star_rating == 5)
vine_5star.count()

3329984

In [62]:
# 5 star_rating with paid votes (Y)
vine_5starY = vine_Y.filter(vine_Y.star_rating == 5)
vine_5starY.count()

8

In [64]:
# 5 star rating with unpaid vote (N)
vine_5starN = vine_N.filter(vine_N.star_rating == 5)
vine_5starN.count()

73039

In [67]:
# Percentage of 5 star rating paid votes
reviews_Y_percent = (vine_5starY.count() / vine_5star.count())*100
print(reviews_Y_percent)

0.0002402413945532471


In [68]:
# Percentage of 5 star rating unpaid votes
reviews_N_percent = (vine_5starN.count() / vine_5star.count())*100
print(reviews_N_percent)

2.193373902096827
